In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import re
import os
import gc

In [2]:
from module.clean_text import clean_text
from module.add_to_df import add_to_df
from module.save_to_csv import save_to_csv

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:


# navLinks=['https://www.annapurnapost.com/category/latest-news/',
#           'https://www.annapurnapost.com/category/editorial/',
#           'https://www.annapurnapost.com/category/apheadline/',
#           'https://www.annapurnapost.com/category/politics/',
#           'https://www.annapurnapost.com/category/opinion/',
#           'https://www.annapurnapost.com/category/social/',
#           'https://www.annapurnapost.com/category/economy/',
#           'https://www.annapurnapost.com/category/sports/',
          
#           ]




In [ ]:

# for navlink in navLinks:
#     driver = webdriver.Chrome()

#     driver.get(navlink)
#     driver.implicitly_wait(10)  
    
#     news_date = driver.find_element(By.CLASS_NAME, 'ap__headerDate').text
#     news_date= news_date.split(',')[0]
#     news_date= news_date.split(' ')
#     # print((f'{news_date[1]+' '+ news_date[0] }'))
#     news_date = f'{news_date[1]+' '+ news_date[0] }'
#     # print(type(news_date))
    
#     main_body= driver.find_element(By.TAG_NAME, "main")
#     main_body= driver.find_element(By.TAG_NAME, "main")
#     find_link = main_body.find_elements(By.TAG_NAME, "a")
#     find_link =list( set(find_link))
    
#     links=[]
#     for link in find_link:
#         # print(link.get_attribute("href"))
#         links.append(link.get_attribute('href'))
#     print(links)
    
#     story_link=[link  for link in links  if link is not None and 'story' in link ]
#     print(len(story_link))

#     story_link = list(set(story_link))
#     print(len(story_link))


#     for link in story_link:
#         try:
#             driver.get(link)
            
#             # Wait for the page to load
#             driver.implicitly_wait(10)  
            
#             #check if the news is of today
#             time = driver.find_element(By.CLASS_NAME, 'date').text
#             time = time.split(',')[0]
#             # if time != news_date:
#             #     # If the news is not from today, skip it
#             #     print(f"Skipping {link} as it is not today's news.")
#             #     continue
            
            
#             ap_news_content=driver.find_element(By.CLASS_NAME, 'ap__news-content')
            
            
#             title= ap_news_content.find_element(By.CLASS_NAME, 'news__title')
#             title=  clean_text(title.text)
#             print(title)
            
#             news_details= ap_news_content.find_element(By.CLASS_NAME, 'news__details')
#             description=  clean_text(news_details.text)
#             print(description)
            
#             print ('\n\n')
            
#             df = add_to_df(title, description)
#             save_to_csv(df , '/var/home/ramrshrcg/Desktop/Python/scrapping/News_csv/AnnapurnaPost',f'{time}_annupurna_post' )
  
        
#         except Exception as e:
#             print(f"Error: {e}")
#             continue


#     driver.close()
#     gc.collect()
#     print("Scraping completed.")
    
    

In [ ]:
def extract_links(navLinks):
    
    for navlink in navLinks:
        print(f"Processing navlink: {navlink}")
        driver = None
        try:
            driver = webdriver.Chrome()
            
            try:
                driver.get(navlink)
                driver.implicitly_wait(10)  
            except Exception as e:
                print(f"Error accessing navlink {navlink}: {e}")
                continue
            
            try:
                # news_date = driver.find_element(By.CLASS_NAME, 'ap__headerDate').text
                news_date = news_date.split(',')[0]
                news_date = news_date.split(' ')
                # news_date = f'{news_date[1]+' '+ news_date[0]}'
                # year= news_date[2]
            except Exception as e:
                print(f"Error extracting news date from {navlink}: {e}")
                news_date = "Unknown"
            
            try:
                main_body = driver.find_element(By.TAG_NAME, "main")
                find_link = main_body.find_elements(By.TAG_NAME, "a")
                find_link = list(set(find_link))
                
                links = []
                for link in find_link:
                    try:
                        href = link.get_attribute('href')
                        links.append(href)
                    except Exception as e:
                        print(f"Error getting href attribute: {e}")
                        continue
                
                print(links)
                
                story_link = [link for link in links if link is not None and 'story' in link]
                print(len(story_link))
                
                story_link = list(set(story_link))
                print(len(story_link))
                
            except Exception as e:
                print(f"Error extracting story links from {navlink}: {e}")
                story_link = []

            for link in story_link:
                try:
                    driver.get(link)
                    
                    # Wait for the page to load
                    driver.implicitly_wait(10)  
                    
                    # Check if the news is of today
                    try:
                        fulltime = driver.find_element(By.CLASS_NAME, 'date').text
                        # print(fulltime)
                        
                        
                        time = fulltime.split(',')[0]
                        
                        # print(time)
                        
                        takingyear = fulltime.split(',')[1]
                        # print(f'takingyear {takingyear}')
                        year= takingyear.split(' ')[1]
                        # print(f' year {year}')
                    except Exception as e:
                        print(f"Error extracting time from {link}: {e}")
                        time = "Unknown"

                    try:
                        ap_news_content = driver.find_element(By.CLASS_NAME, 'ap__news-content')
                        
                        try:
                            title = ap_news_content.find_element(By.CLASS_NAME, 'news__title')
                            title = clean_text(title.text)
                            print(title)
                        except Exception as e:
                            print(f"Error extracting title from {link}: {e}")
                            title = "No title found"
                        
                        try:
                            news_details = ap_news_content.find_element(By.CLASS_NAME, 'news__details')
                            description = clean_text(news_details.text)
                            print(description)
                        except Exception as e:
                            print(f"Error extracting description from {link}: {e}")
                            description = "No description found"
                        
                        print('\n\n')
                        
                        try:
                            df = add_to_df(title, description)
                            save_to_csv(df, f'/var/home/ramrshrcg/Desktop/Python/scrapping/News_csv/AnnapurnaPost/{year}', f'{time}_annupurna_post')
                        except Exception as e:
                            print(f"Error saving data to CSV for {link}: {e}")
                            
                    except Exception as e:
                        print(f"Error finding news content in {link}: {e}")
                        continue
                        
                except Exception as e:
                    print(f"Error processing story link {link}: {e}")
                    continue

        except Exception as e:
            print(f"Error initializing driver for navlink {navlink}: {e}")
            
        finally:
            # Ensure driver is closed even if errors occur
            if driver:
                try:
                    driver.close()
                except Exception as e:
                    print(f"Error closing driver: {e}")
                try:
                    driver.quit()  # Added quit() for complete cleanup
                except Exception as e:
                    print(f"Error quitting driver: {e}")
            
            try:
                gc.collect()
            except Exception as e:
                print(f"Error during garbage collection: {e}")
                
            print("Scraping completed for this navlink.")

SyntaxError: f-string: expecting '}' (4122275136.py, line 20)

In [ ]:

navLinks=[
]
for i in range(1450,1500):
   navLinks.append(f'https://www.annapurnapost.com/category/latest-news/?page={i}')
print(navLinks)
extract_links(navLinks)


In [ ]:
navLinks=[
]
for i in range(3000,3600):
   navLinks.append(f'https://www.annapurnapost.com/category/latest-news/?page={i}')
print(navLinks)
extract_links(navLinks)

अमेरिकाले फेरि गर्यो तीन नेपालीलाई निष्कासन
काठमाडौं : अमेरिकाले फेरि तीन जना नेपालीलाई निष्कासन गरेको छ। गैरकानुनी रूपमा बसोबास गरेको आरोपमा उनीहरूलाई अमेरिकी अध्यागमनले निष्कासन गरेको हो। निष्कासनमा पर्नेमा वली, डाँगी र केसी थरका एकएक जना छन्। स्रोतका अनुसार दुई जनालाई असार १२ गते र एक जनालाई असार ११ गते नेपाल फर्काइएको हो। उनीहरूलाई एयर इन्डियाको जहाजबाट भारत हुँदै काठमाडौं फर्काएको स्रोतको भनाइ छ। अमेरिकाको इमिग्रेशन एन्ड कस्टम इन्फोर्समेन्ट (आइस)का अनुसार अवैधरूपमा बसोबास गरिरहेका उनीहरूलाई अमेरिकाको विभिन्न स्थानबाट पक्राउ परेका थिए। पक्राउ पर्दा उनीहरूसँग कुनै पनि कागजात नरहेको आइस प्रहरीको भनाइ छ। आइस प्रहरीले अमेरिकास्थित नेपाली दूतावाससँग 'ट्राभल डकुमेन्ट'को पहल गरेर उनीहरूलाई नेपाल फर्काएको हो। डोनाल्ड ट्रम्प अमेरिकाको राष्ट्रपति भएपछि त्यहाँका नेपालीसहित अवैधरूपमा बसोबास गरिरहेका अप्रवासीमाथि धरपकड गरिएको छ। ट्रम्प प्रशासनले विभिन्न मितिमा नेपालीहरूलाई नेपाल फर्काउने गरेको छ। ट्रम्प प्रशासनले आप्रवासनसम्बन्धी नयाँ नीति नै लागू गरेर धरपकड गरिरहेको त्यहाँका नेपालीहरूले बताएका

In [ ]:
# driver = webdriver.Chrome()

# driver.get('https://www.annapurnapost.com/category/latest-news/')
# driver.implicitly_wait(10)  

# takingyear = time.split(',')[1]
# year= takingyear.split(' ')[0]
    


In [ ]:


# news_date = driver.find_element(By.CLASS_NAME, 'ap__headerDate').text

# print(news_date)

# alldate= news_date.split(',')[0]
# # print(alldate)

# breakdate= alldate.split(' ')
# print((f'{breakdate[1]+' '+ breakdate[0] }'))
# dayandmonth = f'{breakdate[1]+' '+ breakdate[0]+ ' '+ breakdate[2] }'
# # print(dayandmonth[2])
# # print((dayandmonth))


In [ ]:
# driver.get('https://www.annapurnapost.com/story/481387/')
            
# # Wait for the page to load
# driver.implicitly_wait(10)  
            
# #check if the news is of today
# time = driver.find_element(By.CLASS_NAME, 'date').text
# time = time.split(',')[0]


In [ ]:
# print((time))

In [ ]:
# if time != news_date:
# # If the news is not from today, skip it
#     print(f"Skipping  as it is not today's news.")

# is_today_news = time == news_date
# is_today_news
   

In [ ]:
# find_link = main_body.find_elements(By.TAG_NAME, "a")


In [ ]:
# find_link =list( set(find_link))

In [ ]:
# links=[]
# for link in find_link:
#     # print(link.get_attribute("href"))
#     links.append(link.get_attribute('href'))
# print(links)

In [ ]:
# story_link=[link  for link in links  if link is not None and 'story' in link ]
# print(len(story_link))

# story_link = list(set(story_link))
# print(len(story_link))


In [ ]:
# for link in story_link:
#     try:
#         driver.get(link)
        
#         # Wait for the page to load
#         driver.implicitly_wait(10)  
        
#         #check if the news is of today
#         time = driver.find_element(By.CLASS_NAME, 'date').text
#         time = time.split(',')[0]
#         if time != news_date:
#             # If the news is not from today, skip it
#             print(f"Skipping {link} as it is not today's news.")
#             continue
        
        
#         ap_news_content=driver.find_element(By.CLASS_NAME, 'ap__news-content')
        
        
#         title= ap_news_content.find_element(By.CLASS_NAME, 'news__title')
#         title=  clean_text(title.text)
#         print(title)
        
#         news_details= ap_news_content.find_element(By.CLASS_NAME, 'news__details')
#         description=  clean_text(news_details.text)
#         print(description)
        
#         print ('\n\n')
        
#         df = add_to_df(title, description)
#         save_to_csv(df , '/var/home/ramrshrcg/Desktop/Python/scrapping/News_csv/AnnapurnaPost','annupurna_post' )
        
        
        
        
        
        
        
        
#     except Exception as e:
#         print(f"Error: {e}")
#         continue

In [ ]:
gc.enable()

In [ ]:
# response = requests.get("https://www.annapurnapost.com/category/latest-news/")
# soup = BeautifulSoup(response.content, "html.parser")
# print(soup.prettify())

In [ ]:
# category__banner = soup.find("div" ,class_ ='category__banner')
# print((category__banner))

In [ ]:
# driver.close()